# Acme Corp Data Agents with Strands SDK

This notebook demonstrates how to create AI agents using the Strands SDK to interact with Acme Corp's MCP servers for:
- User Details
- Streaming Analytics
- Ad Campaign Data

## Prerequisites

1. Install the Strands SDK: `pip install strands-sdk`
2. Start the MCP servers in separate terminals:
   - `cd acme-corp-data/mcp-servers/user-details-mcp && npm start`
   - `cd acme-corp-data/mcp-servers/streaming-analytics-mcp && npm start`
   - `cd acme-corp-data/mcp-servers/ad-campaign-mcp && npm start`

In [ ]:
# Import required libraries
from strands import Agent, MCPClient
import asyncio
import json
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set up plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

## 1. Initialize MCP Clients

First, we'll create MCP clients for each of our data sources.

In [ ]:
# Initialize MCP clients
user_details_client = MCPClient(
    name="user-details",
    command="node",
    args=["acme-corp-data/mcp-servers/user-details-mcp/index.js"]
)

streaming_analytics_client = MCPClient(
    name="streaming-analytics",
    command="node",
    args=["acme-corp-data/mcp-servers/streaming-analytics-mcp/index.js"]
)

ad_campaign_client = MCPClient(
    name="ad-campaign",
    command="node",
    args=["acme-corp-data/mcp-servers/ad-campaign-mcp/index.js"]
)

# Start all clients
async def start_clients():
    await user_details_client.start()
    await streaming_analytics_client.start()
    await ad_campaign_client.start()
    print("All MCP clients started successfully!")

# Run the async function
await start_clients()

## 2. Create Specialized Agents

We'll create different agents for different business functions.

In [ ]:
# Customer Insights Agent
customer_insights_agent = Agent(
    name="Customer Insights Analyst",
    role="Analyze customer behavior and provide insights",
    mcp_clients=[user_details_client, streaming_analytics_client],
    system_prompt="""You are a customer insights analyst for Acme Corp. 
    You have access to user details and streaming analytics data.
    Provide actionable insights about customer segments, viewing behavior, and engagement patterns.
    Always support your insights with data."""
)

# Marketing Performance Agent
marketing_agent = Agent(
    name="Marketing Performance Specialist",
    role="Optimize ad campaigns and marketing spend",
    mcp_clients=[ad_campaign_client, user_details_client],
    system_prompt="""You are a marketing performance specialist for Acme Corp.
    You have access to ad campaign data and user demographics.
    Focus on ROI optimization, campaign performance, and targeting recommendations.
    Provide specific, actionable recommendations backed by data."""
)

# Business Intelligence Agent
bi_agent = Agent(
    name="Business Intelligence Analyst",
    role="Provide comprehensive business insights",
    mcp_clients=[user_details_client, streaming_analytics_client, ad_campaign_client],
    system_prompt="""You are a senior business intelligence analyst with access to all Acme Corp data.
    Synthesize insights across user behavior, content performance, and marketing effectiveness.
    Identify cross-functional opportunities and strategic recommendations.
    Present findings in a clear, executive-ready format."""
)

print("Agents created successfully!")

## 3. Customer Insights Analysis

Let's use the Customer Insights Agent to analyze user segments and behavior.

In [ ]:
# Analyze customer segments
segment_analysis = await customer_insights_agent.query(
    """Analyze our customer base:
    1. What are the main customer segments by subscription type and demographics?
    2. Which segments show the highest engagement with our streaming content?
    3. Are there any underserved segments we should target?
    """
)

print(segment_analysis)

In [ ]:
# Deep dive into viewing patterns
viewing_patterns = await customer_insights_agent.query(
    """Analyze viewing patterns:
    1. What are the peak viewing hours across different user segments?
    2. Which content genres drive the most engagement?
    3. What's the correlation between subscription type and content preferences?
    4. Identify users with declining engagement who might churn.
    """
)

print(viewing_patterns)

## 4. Marketing Campaign Optimization

Now let's use the Marketing Agent to optimize our ad campaigns.

In [ ]:
# Campaign performance analysis
campaign_analysis = await marketing_agent.query(
    """Analyze our current ad campaigns:
    1. Which campaigns have the best ROI?
    2. What are the main issues with underperforming campaigns?
    3. How do attribution paths differ across platforms?
    4. Provide specific optimization recommendations for our top 3 campaigns.
    """
)

print(campaign_analysis)

In [ ]:
# Targeting optimization
targeting_recommendations = await marketing_agent.query(
    """Based on our user data and campaign performance:
    1. Which user segments should we target more aggressively?
    2. Are there high-value segments we're missing?
    3. How should we adjust our budget allocation across platforms?
    4. What creative strategies would resonate with our key segments?
    """
)

print(targeting_recommendations)

## 5. Executive Business Intelligence Report

Let's use the BI Agent to create a comprehensive business report.

In [ ]:
# Comprehensive business analysis
executive_report = await bi_agent.query(
    """Create an executive summary covering:
    
    1. Business Health Metrics:
       - Total active users and growth trends
       - Revenue by subscription tier
       - Customer acquisition cost vs lifetime value
    
    2. Content Performance:
       - Top performing content and genres
       - User engagement metrics and trends
       - Content ROI analysis
    
    3. Marketing Effectiveness:
       - Overall marketing ROI
       - Channel performance comparison
       - Customer acquisition funnel analysis
    
    4. Strategic Recommendations:
       - Top 3 growth opportunities
       - Risk factors to monitor
       - Resource allocation recommendations
    
    Present this as an executive-ready report with clear metrics and actionable insights.
    """
)

print(executive_report)

## 6. Interactive Analysis Examples

Here are some interactive analysis examples you can try.

In [ ]:
# Example 1: Find similar users for targeted campaigns
similar_users_analysis = await marketing_agent.query(
    """Find users similar to our top converters from campaign 'camp_001':
    1. Identify the characteristics of users who converted
    2. Find similar users who haven't been targeted yet
    3. Estimate the potential ROI of targeting these users
    """
)

print(similar_users_analysis)

In [ ]:
# Example 2: Content recommendation strategy
content_strategy = await customer_insights_agent.query(
    """Develop a content recommendation strategy:
    1. Which users are most likely to upgrade to premium based on viewing patterns?
    2. What content should we recommend to increase engagement?
    3. How can we reduce churn for users with declining engagement?
    """
)

print(content_strategy)

In [ ]:
# Example 3: Cross-sell opportunities
cross_sell_analysis = await bi_agent.query(
    """Identify cross-sell opportunities:
    1. Which free users show behavior patterns similar to premium subscribers?
    2. What's the optimal timing for upgrade offers based on user journey?
    3. Which marketing channels are most effective for upgrade campaigns?
    4. Calculate potential revenue impact of a targeted upgrade campaign.
    """
)

print(cross_sell_analysis)

## 7. Custom Analysis Function

Create a reusable function for common analyses.

In [ ]:
async def cohort_analysis(agent, cohort_definition, metrics):
    """
    Perform cohort analysis on user segments
    
    Args:
        agent: The Strands agent to use
        cohort_definition: How to define the cohort (e.g., "users who joined in January")
        metrics: List of metrics to analyze (e.g., ["retention", "engagement", "revenue"])
    """
    query = f"""
    Perform cohort analysis for {cohort_definition}:
    
    Analyze the following metrics:
    {', '.join(metrics)}
    
    Provide:
    1. Cohort size and characteristics
    2. Metric trends over time
    3. Comparison with other cohorts
    4. Actionable insights for this cohort
    """
    
    return await agent.query(query)

# Example usage
premium_cohort_analysis = await cohort_analysis(
    customer_insights_agent,
    "Premium subscribers who joined in the last 30 days",
    ["content engagement", "viewing hours", "genre preferences"]
)

print(premium_cohort_analysis)

## 8. Multi-Agent Collaboration

Example of multiple agents working together on a complex analysis.

In [ ]:
# Step 1: Customer Insights Agent identifies high-value segments
high_value_segments = await customer_insights_agent.query(
    "Identify the top 3 most valuable customer segments based on engagement and viewing patterns."
)

print("Step 1 - High Value Segments:")
print(high_value_segments)
print("\n" + "="*50 + "\n")

# Step 2: Marketing Agent develops targeting strategy
targeting_strategy = await marketing_agent.query(
    f"""Based on these high-value segments identified:
    {high_value_segments}
    
    Develop a targeted marketing strategy:
    1. Which campaigns should we create or modify?
    2. What's the optimal budget allocation?
    3. Which platforms should we prioritize?
    """
)

print("Step 2 - Targeting Strategy:")
print(targeting_strategy)
print("\n" + "="*50 + "\n")

# Step 3: BI Agent projects business impact
business_impact = await bi_agent.query(
    f"""Based on the targeting strategy:
    {targeting_strategy}
    
    Project the business impact:
    1. Expected ROI and revenue increase
    2. User acquisition and retention improvements
    3. Overall impact on business KPIs
    4. Implementation timeline and resource requirements
    """
)

print("Step 3 - Business Impact:")
print(business_impact)

## 9. Cleanup

Don't forget to clean up the MCP connections when done.

In [ ]:
# Close all MCP client connections
async def cleanup():
    await user_details_client.stop()
    await streaming_analytics_client.stop()
    await ad_campaign_client.stop()
    print("All MCP clients stopped successfully!")

await cleanup()

## Summary

This notebook demonstrated how to:

1. **Connect to MCP Servers**: Initialize connections to multiple data sources
2. **Create Specialized Agents**: Build agents with specific roles and expertise
3. **Perform Complex Analyses**: Use natural language to query and analyze data
4. **Multi-Agent Collaboration**: Coordinate multiple agents for comprehensive insights
5. **Generate Business Intelligence**: Create executive-ready reports and recommendations

### Key Benefits:

- **Natural Language Interface**: Query data using plain English
- **Contextual Understanding**: Agents understand business context and objectives
- **Cross-Functional Analysis**: Combine data from multiple sources seamlessly
- **Actionable Insights**: Get specific recommendations, not just data

### Next Steps:

1. Extend the MCP servers with more sophisticated analytics tools
2. Create more specialized agents for specific business functions
3. Build automated reporting workflows
4. Integrate with real-time data streams for live monitoring
5. Add visualization capabilities to the agents